# Do the larger quad tree grid cells correspond to non-residential land uses? 

This notebook has been used to answer this referee question.

In [1]:
# Install package utils
import sys
!{sys.executable} -m pip install --user -q -q -e ../

In [2]:
from mobitic_utils.db import engine
with engine.connect() as connection:
        res = connection.execute("SELECT SUM(p.population) FROM filosofi p").fetchall()

In [21]:
query = """
WITH data as (
	   (SELECT p.population as density,
	   SQRT(ST_AREA(p.geometry)) as tile_size,
	   g."x-y" as tile FROM filosofi as p, grille g WHERE p."x-y"=g."x-y")
	   UNION
	   (SELECT COALESCE(0) as density,
	   SQRT(ST_AREA(g.geometry)) as tile_size,
	   g."x-y" as tile FROM grille g WHERE g."x-y" not in (SELECT p."x-y" FROM filosofi p))
	   )
SELECT AVG(density), tile_size, count(DISTINCT tile) FROM data GROUP BY tile_size
"""

with engine.connect() as connection:
        res = connection.execute(query).fetchall()

In [23]:
import pandas as pd
df = pd.DataFrame([list(x) for x in res], columns = ['AVG_DENSITY','TILE_SIDE','DISTINCT_TILES']).sort_values('TILE_SIDE')
df 

,AVG_DENSITY,TILE_SIDE,DISTINCT_TILES
0,23208.948864,100.0,2816
1,15894.860906,200.0,4062
2,5666.538057,400.0,8796
3,1648.903879,800.0,12721
4,278.740541,1600.0,20743
5,61.119656,3200.0,28339
6,27.031261,6400.0,5024
7,10.000610,12800.0,2


In [24]:
df.style.to_latex()

'\\begin{tabular}{lrrr}\n & AVG_DENSITY & TILE_SIDE & DISTINCT_TILES \\\\\n0 & 23208.948864 & 100.000000 & 2816 \\\\\n1 & 15894.860906 & 200.000000 & 4062 \\\\\n2 & 5666.538057 & 400.000000 & 8796 \\\\\n3 & 1648.903879 & 800.000000 & 12721 \\\\\n4 & 278.740541 & 1600.000000 & 20743 \\\\\n5 & 61.119656 & 3200.000000 & 28339 \\\\\n6 & 27.031261 & 6400.000000 & 5024 \\\\\n7 & 10.000610 & 12800.000000 & 2 \\\\\n\\end{tabular}\n'

In [25]:
df['DISTINCT_TILES'].sum()

82503

In [31]:
import numpy as np
np.sum(df['TILE_SIDE'].apply(lambda x : x**2)*df['DISTINCT_TILES'])/(1000**2)

559143.6